In [1]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
import os
import time
from datetime import datetime
import random
import hashlib

KEY = os.urandom(16)

def generate_timestamp():
    timestamp = time.time()
    timestamp_ms = round(timestamp,3)   
    date = datetime.fromtimestamp(timestamp_ms)

    timestamp_int = int(timestamp_ms * 1000)
    ts = timestamp_int.to_bytes(16, byteorder='big') 
    return str(date), hashlib.md5(ts).digest()

In [2]:
def retrieve_hash(text_date):
  date = datetime.strptime(text_date, "%Y-%m-%d %H:%M:%S.%f")
  timestamp = date.timestamp()
  timestamp_ms = round(timestamp,3)
  timestamp_int = int(timestamp_ms * 1000)
  ts = timestamp_int.to_bytes(16, byteorder='big')
  return hashlib.md5(ts).digest()

In [3]:
with open('output.txt', 'r') as f:
  lines = f.readlines()

In [4]:
text_date, text_time, text_cipher = lines[0].strip().split(' ')
text_date = text_date + ' ' + text_time
print("Date:", text_date)
print("Cypher:", text_cipher)

flag_date, flag_time, flag_cipher = lines[1].strip().split(' ')
flag_date = flag_date + ' ' + flag_time
print("Date:", flag_date)
print("Cypher:", flag_cipher)

Date: 2025-03-10 09:50:07.974000
Cypher: ac0720038488a5f6f149cf0f3a41e31ffff996363fa6a0cf493c2a2998ad0bddfab0422efd4df201bf05ede07926ce3988015bb35a717504d03db4a0ac52d67c8f006b9e662f0db0a6644484d210eb33d3ce54d9565c10bb2fcc08d8db5e87d96d576418
Date: 2025-03-10 09:50:10.975000
Cypher: b571b8eb470f7c3b466618af91cb111184d73bfca56c3f1ddc44c5845f789f8584d7a6fe60d3328b386a0044b8e1164bb810f97398e8b98d280903220dc33452be73e90fefa9


In [5]:
text_hash = retrieve_hash(text_date)
print("Text hash:", text_hash)

flag_hash = retrieve_hash(flag_date)
print("Flag hash:", flag_hash)

Text hash: b'\xa5l\xf4\xcc5\xa1K\xf3&f\xedl\xe9\x88\x04\x01'
Flag hash: b'\x84=P\x1a\xae)\xd3)\x82]\x17\xd0\x12_\xcdM'


In [7]:
test = b"Capybara friends, mission accomplished! We've caused a blackout, let's meet at the bar to celebrate!"

In [8]:
text_cipher_hex = bytes.fromhex(text_cipher)
text_cipher_blocks = [text_cipher_hex[i:i+16] for i in range(0, len(text_cipher_hex), 16)]

text_cipher_xor_blocks = []
for block in text_cipher_blocks:
  block_with_xor = bytes(a ^ b for a, b in zip(block, text_hash))
  text_cipher_xor_blocks.append(block_with_xor)

text_cipher_xor_blocks

[b'\tk\xd4\xcf\xb1)\xee\x05\xd7/"c\xd3\xc9\xe7\x1e',
 b'Z\x95b\xfa\n\x07\xeb<oZ\xc7Eq%\x0f\xdc',
 b'_\xdc\xb6\xe2\xc8\xec\xb9\xf2\x99c\x00\x8c\x90\xae\xca8',
 b'-m\xaf\x7fo\xd0>\xf7\xf6[Y\xccE\xda\xd2}',
 b'*l\x9fRS\x8eFC\x80\x02\xa9\xe8;\x98\xef2',
 b'v\xa2\xa0\x15c\xfd[H\t\xaa\xe5\xb42\xd6\x83\xd8',
 b'\xc8;\x90\xd4']

In [9]:
flag_cipher_hex = bytes.fromhex(flag_cipher)
flag_cipher_blocks = [flag_cipher_hex[i:i+16] for i in range(0, len(flag_cipher_hex), 16)]

flag_cipher_xor_blocks = []
for block in flag_cipher_blocks:
  block_with_xor = bytes(a ^ b for a, b in zip(block, flag_hash))
  flag_cipher_xor_blocks.append(block_with_xor)

flag_cipher_xor_blocks

[b'1L\xe8\xf1\xe9&\xaf\x12\xc4;\x0f\x7f\x83\x94\xdc\\',
 b"\x00\xeak\xe6\x0bE\xec4^\x19\xd2TM'R\xc8",
 b'\x00\xea\xf6\xe4\xce\xfa\xe1\xa2\xba7\x17\x94\xaa\xbe\xdb\x06',
 b'<-\xa9i6\xc1j\xa4\xaaT\x14\xf2\x1f\x9c\xf9\x1f',
 b':N\xb9\x15A\x80']

In [11]:
# Pad the test message to be a multiple of 16 bytes
test += b'\x00' * (16 - len(test) % 16)

# divide the test mesasge in blocks of 16 bytes
test_blocks = [test[i:i+16] for i in range(0, len(test), 16)]

E_K = []
# For each block in test_blocks, xor it with the corresponding block in text_cipher_xor_blocks
for i in range(len(test_blocks)):
  E_K.append(bytes(a ^ b for a, b in zip(test_blocks[i], text_cipher_xor_blocks[i])))

E_K

[b'J\n\xa4\xb6\xd3H\x9cd\xf7IP\n\xb6\xa7\x83m',
 b'v\xb5\x0f\x93yt\x82S\x01z\xa6&\x12Jb\xac',
 b"3\xb5\xc5\x8a\xad\x88\x98\xd2\xce\x06'\xfa\xf5\x8e\xa9Y",
 b'X\x1e\xca\x1bO\xb1\x1e\x95\x9a::\xa7*\xaf\xa6Q',
 b'\n\x00\xfa&t\xfdf.\xe5g\xdd\xc8Z\xec\xcfF',
 b'\x1e\xc7\x80w\x02\x8f{<f\x8a\x86\xd1^\xb3\xe1\xaa',
 b'\xa9O\xf5\xf5']

In [12]:
# Xor the E_K blocks with the flag_cipher_xor_blocks
flag_blocks = []
for i in range(len(flag_cipher_xor_blocks)):
  flag_blocks.append(bytes(a ^ b for a, b in zip(E_K[i], flag_cipher_xor_blocks[i])))

flag_blocks

[b'{FLG:n3v3r_u53_1',
 b'v_dur1ng_ctr_m0d',
 b'3_3ncrypt10n_0r_',
 b'd3crypt10n.U53_N',
 b'0NC35}']

In [ ]:
flag = b''.join(flag_blocks)
print(flag.decode())


{FLG:n3v3r_u53_1v_dur1ng_ctr_m0d3_3ncrypt10n_0r_d3crypt10n.U53_N0NC35}
